# 6-Month Circulation By Branch and Item Type

In [1]:
import requests
import pandas as pd
import altair as alt
from datetime import date
from dateutil.relativedelta import relativedelta
import zipfile

In [2]:
# set the API base URL
base_url = "https://collection-analysis.cincy.pl/current_collection/"

# create a requests session for connection pooling
session = requests.session()

In [3]:
# list comprehension to get the list of branches ...
branches = [
    branch[2]
    for branch
    in (
        session.get(base_url+'branch_name.json').json()['rows']
    )
    if branch[2] != ''
]

In [4]:
# create the dataframe where we'll store the results
df = pd.DataFrame(index=None)

# this is the API endpoint where our current collection results will come from
next_url = base_url+'circ_agg_branch_view.json'

# work through the results (the endpoint will return `None` when no more rows)
while next_url is not None:
  r = session.get(next_url)

  # concatinate the results into the DataFrame as we go
  df = pd.concat(
      [
          df,
          pd.DataFrame(
              r.json()['rows'],
              columns=r.json()['columns'],
              index=None
          )
      ],
  )

  # get the next url (results are paginated)
  if 'next_url' in r.json().keys():
    next_url = r.json()['next_url']
    print('.', end='')
  else:
    break

...............................................................

In [5]:
# branch_name = 'Cheviot'
html_files = []

for branch_name in branches:

  # assume that this report is going to be run at any time...
  # so, assume that we have 5 months of complete data + the current month
  six_months_ago = (date.today() + relativedelta(months=-5)).strftime('%Y-%m')

  # filter the DataFrame by the parameters above
  source = df[
      (df['transaction_month'] >= six_months_ago)
      & (df['branch_name'] == branch_name)
  ]

  # create the chart for the branch
  chart = alt.Chart(source).mark_bar().encode(
      x=alt.X('sum_transactions'),
      y=alt.Y('itype_name', sort='-x'),
      color='checkout_type',
      tooltip=['branch_name', 'transaction_month', 'itype_name', 'sum_transactions']
  ).facet(
      'transaction_month',
      columns=3
  ).properties(
      title=f'{branch_name} Item Type Circulations 6 Months (generated: {date.today()})'
  )

  # create a zip file of all the html output (append to the zip file as we go)
  with zipfile.ZipFile(f'{date.today()}_6-months_circ_by_branch.zip', mode='a') as z:
    z.writestr(
        f"{branch_name}.html",
        chart.to_html().encode('utf-8')
    )

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:219: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  warnings.warn(
